In [1]:
import random
from Data_import import data_frame
from model_evaluation import *
from _xgbod import hazard_wind_detection_XGBOD
np.random.seed(1)
random.seed(1)

Data = data_frame()
calm_train = Data.calm_train
pilot_train = Data.pilot_semi_learning_generator(semi_ratio = 1)
X_unlabel = Data.unlabeled_data_v_r[::500,::]#KNN can cause a lot of computation load
Y_unlabel = np.zeros((X_unlabel.shape[0],1))
train_x = np.vstack((calm_train, pilot_train))#labeled data
train_y = np.hstack((np.zeros(calm_train.shape[0]), np.zeros(pilot_train.shape[0]) + 1))

model = hazard_wind_detection_XGBOD()
model.train_base_unsupervise_classifiers(X_unlabel)

In [2]:
import numpy as np
from models.utility import print_baseline
from xgboost.sklearn import XGBClassifier
for count_imbalance, beta_imbalance in enumerate(np.array([1,10,100,200,400,600,800,1000])):
    calm_test_x = Data.calm_test_generator(calm_ratio=beta_imbalance)
    test_x = np.vstack((calm_test_x, Data.pilot_test_v_r))
    test_y = np.hstack((np.zeros(calm_test_x.shape[0]), np.zeros(Data.pilot_test_v_r.shape[0]) + 1))
    X_train_new, test_x_new, roc_list = model.unsupervised_classifiers_decision_functions(train_x, test_x, test_y)
    X_train_all = np.concatenate((train_x, X_train_new), axis=1)
    test_x_all = np.concatenate((test_x, test_x_new), axis=1)

    # get the results of baselines
#     print_baseline(test_x_new, test_y, roc_list)
    clf = XGBClassifier()
    # fully supervised
    clf.fit(train_x, train_y.ravel())
    y_pred = clf.predict_proba(test_x)
    supervised_AUC, _ = calculate_metrics(decision_values=y_pred[:, 1], truth=test_y, hazard_id=1)

    # unsupervised
    clf.fit(X_train_new, train_y.ravel())
    y_pred = clf.predict_proba(test_x_new)
    unsupervised_AUC, _ = calculate_metrics(decision_values=y_pred[:, 1], truth=test_y, hazard_id=1)

    # semi-supervised
    clf.fit(X_train_all, train_y.ravel())
    y_pred = clf.predict_proba(test_x_all)
    semisupervised_AUC, _ = calculate_metrics(decision_values=y_pred[:, 1], truth=test_y, hazard_id=1)

    print(f"=====================beta_imbalance = {beta_imbalance}=====================")
    print(f'superviseAUC (XGBoost): {supervised_AUC}')
    # print(f'unsuperviseAUC: {unsupervised_AUC}')
    print(f'semi-superviseAUC: {semisupervised_AUC}')


=====================beta_imbalance = 1=====================
superviseAUC (XGBoost): 0.8851977040816327
semi-superviseAUC: 0.9985841836734695
=====================beta_imbalance = 10=====================
superviseAUC (XGBoost): 0.8787576530612244
semi-superviseAUC: 0.9930031887755102
=====================beta_imbalance = 100=====================
superviseAUC (XGBoost): 0.8681908801020408
semi-superviseAUC: 0.9687598214285714
=====================beta_imbalance = 200=====================
superviseAUC (XGBoost): 0.8585384885204081
semi-superviseAUC: 0.9418404017857144
=====================beta_imbalance = 400=====================
superviseAUC (XGBoost): 0.8421870854591837
semi-superviseAUC: 0.8744178411989796
=====================beta_imbalance = 600=====================
superviseAUC (XGBoost): 0.8282520620748299
semi-superviseAUC: 0.8102776573129252
=====================beta_imbalance = 800=====================
superviseAUC (XGBoost): 0.8148447305484694
semi-superviseAUC: 0.759352694515

In [5]:
XGBoost_AUC = [0.8851977040816327, 0.8787576530612244, 0.8681908801020408, 0.8585384885204081, 
               0.8421870854591837, 0.8282520620748299, 0.8148447305484694, 0.8017956696428571]
XGBOD_AUC =   [0.9985841836734695, 0.9930031887755102, 0.9687598214285714, 0.9418404017857144,
               0.8744178411989796, 0.8102776573129252, 0.7593526945153062, 0.7196182971938776]